In [3]:
pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 2.9 MB/s eta 0:00:00


In [69]:
# Ejecución del pipeline principal
# Asegurarse de que df y target_column estén definidos correctamente
df = pd.read_csv('/content/ObesityDataSet_raw_and_data_sinthetic.csv')
df.head(10)


,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II
5,Male,29.0,1.62,53.0,no,yes,2.0,3.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Automobile,Normal_Weight
6,Female,23.0,1.50,55.0,yes,yes,3.0,3.0,Sometimes,no,2.0,no,1.0,0.0,Sometimes,Motorbike,Normal_Weight
7,Male,22.0,1.64,53.0,no,no,2.0,3.0,Sometimes,no,2.0,no,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
8,Male,24.0,1.78,64.0,yes,yes,3.0,3.0,Sometimes,no,2.0,no,1.0,1.0,Frequently,Public_Transportation,Normal_Weight
9,Male,22.0,1.72,68.0,yes,yes,2.0,3.0,Sometimes,no,2.0,no,1.0,1.0,no,Public_Transportation,Normal_Weight


In [49]:

df_preprocessed, preprocessor = preprocessing_pipeline(df)
preprocessor

ColumnTransformer(transformers=[('log',
                                 Pipeline(steps=[('log',
                                                  FunctionTransformer(func=<ufunc 'log'>,
                                                                      validate=True)),
                                                 ('scaler', StandardScaler())]),
                                 ['Age']),
                                ('num',
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 Index(['Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE'], dtype='object')),
                                ('cat',
                                 Pipeline(steps=[('binary',
                                                  BinaryEncoder(handle_unknown='ignore'))]),
                                 ['Gender', 'family_history_with_overweight',
                                  'FAVC', 'CAEC', 'SMOKE', 'SCC', 'MTRANS',
                                  'NObeyesdad']),
                                ('ord',
                                 Pipeline(steps=[('ordinary',
                                                  OrdinalEncoder())]),
                                 ['FCVC', 'NCP', 'CH2O', 'FAF', 'TUE',
                                  'CALC'])])

ML PIPE

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import joblib



# Función para guardar todo el pipeline (preprocesamiento + modelo)
def save_best_pipeline(pipeline, filename="best_pipeline.pkl"):
    joblib.dump(pipeline, filename)
    print(f"Pipeline completo guardado en {filename}")

# Separar X (características) e Y (objetivo)
def separate_X_y(df, target_column):
    X = df.drop(columns=[target_column])
    y = df[target_column]
    return X, y

# Identificar variables categóricas, numéricas y ordinales
def identify_variable_types(X):
    numericas = X.select_dtypes(include=['int64', 'float64']).columns
    categoricas = X.select_dtypes(include=['object']).columns
    ordinales = ['FCVC', 'NCP', 'CH2O', 'FAF', 'TUE', 'CALC']  # Variables ordinales conocidas

    ordinales = [col for col in ordinales if col in X.columns]
    categoricas = [col for col in categoricas if col not in ordinales]

    return numericas, categoricas, ordinales

# Preprocesamiento
def create_preprocessing_pipeline(numericas, categoricas, ordinales):
    pip_num = Pipeline(steps=[('scaler', StandardScaler())])

    # Manejar categorías desconocidas en el OneHotEncoder y OrdinalEncoder
    pip_cat = Pipeline(steps=[('onehot', OneHotEncoder(drop='first', handle_unknown='ignore'))])
    pip_ord = Pipeline(steps=[('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))])

    # Combinar los pipelines en un ColumnTransformer
    pipeline_preprocesada = ColumnTransformer(transformers=[
        ('num', pip_num, numericas),
        ('cat', pip_cat, categoricas),
        ('ord', pip_ord, ordinales)
    ], remainder='passthrough')

    return pipeline_preprocesada

# Pipeline final
def main_pipeline(df, target_column):
    X, y = separate_X_y(df, target_column)
    numericas, categoricas, ordinales = identify_variable_types(X)

    # Crear pipeline de preprocesamiento
    preprocessing_pipeline = create_preprocessing_pipeline(numericas, categoricas, ordinales)

    # Dividir en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Entrenar modelos con búsqueda de hiperparámetros
    best_model = train_with_hyperparameter_tuning(preprocessing_pipeline, X_train, y_train)

    # Crear el pipeline completo (preprocesamiento + mejor modelo)
    full_pipeline = Pipeline(steps=[
        ('preprocessing', preprocessing_pipeline),
        ('model', best_model)
    ])

    # Evaluar y guardar el mejor pipeline completo
    evaluate_and_save_pipeline(full_pipeline, X_test, y_test)

# Entrenamiento con GridSearchCV
def train_with_hyperparameter_tuning(preprocessing_pipeline, X_train, y_train):
    models = {
        'Logistic Regression': LogisticRegression(max_iter=1000),
        'Random Forest': RandomForestClassifier(),
        'Gradient Boosting': GradientBoostingClassifier(),
        'KNeighbors': KNeighborsClassifier(),
        'DecisionTree': DecisionTreeClassifier(),
        'AdaBoost': AdaBoostClassifier()
    }

    param_grids = {
        'Logistic Regression': {'C': [0.1, 1, 10]},
        'Random Forest': {'n_estimators': [50, 100], 'max_depth': [5, 10]},
        'Gradient Boosting': {'n_estimators': [50, 100], 'learning_rate': [0.1, 0.01]},
        'KNeighbors': {'n_neighbors': [3, 5, 7]},
        'DecisionTree': {'max_depth': [5, 10, None]},
        'AdaBoost': {'n_estimators': [50, 100], 'learning_rate': [0.1, 1]}
    }

    best_model = None
    best_score = 0
    for model_name, model in models.items():
        grid_search = GridSearchCV(model, param_grids[model_name], cv=5, scoring='accuracy', n_jobs=-1, verbose=1)
        grid_search.fit(preprocessing_pipeline.fit_transform(X_train), y_train)
        print(f"Mejores hiperparámetros para {model_name}: {grid_search.best_params_}")

        if grid_search.best_score_ > best_score:
            best_score = grid_search.best_score_
            best_model = grid_search.best_estimator_

    return best_model

# Evaluar y guardar el pipeline completo
def evaluate_and_save_pipeline(pipeline, X_test, y_test):
    y_pred = pipeline.predict(X_test)
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print("Matriz de Confusión:")
    print(confusion_matrix(y_test, y_pred))
    print("Reporte de Clasificación:")
    print(classification_report(y_test, y_pred))

    # Generar gráficos de Gain y Lift
    gain_lift_chart(pipeline, X_test, y_test)

    # Guardar el pipeline completo
    save_best_pipeline(pipeline)

# Leer archivo CSV
#df = pd.read_csv('/content/ObesityDataSet_raw_and_data_sinthetic.csv')

# Ejecutar el pipeline principal con el target deseado
main_pipeline(df, target_column='NObeyesdad')


Fitting 5 folds for each of 3 candidates, totalling 15 fits


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Mejores hiperparámetros para Logistic Regression: {'C': 0.1}
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Mejores hiperparámetros para Random Forest: {'max_depth': 10, 'n_estimators': 100}
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Mejores hiperparámetros para Gradient Boosting: {'learning_rate': 0.1, 'n_estimators': 100}
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Mejores hiperparámetros para KNeighbors: {'n_neighbors': 3}
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Mejores hiperparámetros para DecisionTree: {'max_depth': None}
Fitting 5 folds for each of 4 candidates, totalling 20 fits


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Mejores hiperparámetros para AdaBoost: {'learning_rate': 1, 'n_estimators': 50}
Accuracy: 0.9433
Matriz de Confusión:
[[54  2  0  0  0  0  0]
 [ 4 53  0  0  0  5  0]
 [ 0  0 71  5  0  1  1]
 [ 0  0  1 57  0  0  0]
 [ 0  0  0  0 63  0  0]
 [ 0  3  0  0  0 52  1]
 [ 0  0  0  0  0  1 49]]
Reporte de Clasificación:
                     precision    recall  f1-score   support

Insufficient_Weight       0.93      0.96      0.95        56
      Normal_Weight       0.91      0.85      0.88        62
     Obesity_Type_I       0.99      0.91      0.95        78
    Obesity_Type_II       0.92      0.98      0.95        58
   Obesity_Type_III       1.00      1.00      1.00        63
 Overweight_Level_I       0.88      0.93      0.90        56
Overweight_Level_II       0.96      0.98      0.97        50

           accuracy                           0.94       423
          macro avg       0.94      0.95      0.94       423
       weighted avg       0.94      0.94      0.94       423

Revisión de v

In [17]:
loaded_pipeline = joblib.load("best_pipeline.pkl")
predictions = loaded_pipeline.predict(df)


In [18]:
predictions

array(['Normal_Weight', 'Normal_Weight', 'Normal_Weight', ...,
       'Obesity_Type_III', 'Obesity_Type_III', 'Obesity_Type_III'],
      dtype=object)

Refactor